In [ ]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

In [ ]:
# Data ingest
data_url = "https://drive.google.com/uc?id=1_jYVXj7mt8Zzpjn8WGI111G-kWRTbfjU"
df = pd.read_csv(data_url,index_col='date',parse_dates=['date'])
df.info()


In [ ]:
# Downsampling
df = df.resample('8h').median()
df = df.dropna()

In [ ]:
# Data statistics
df.describe().T

In [ ]:
# Plots
tags = ['plant.feed.silica.comp','plant.filters.product.silica.comp']
df[tags].plot(subplots=True,figsize=(16,4))

In [ ]:
# Feed
feed_variables = [name for name in df.columns if 'feed' in name or 'sump01' in name]
sns.pairplot(data=df,vars=feed_variables,corner=True,kind='hist',diag_kind='kde')

In [ ]:
# Feed and product compositions
assay_variables = [name for name in df.columns if 'iron' in name or 'silica' in name]
sns.pairplot(data=df,vars=assay_variables,corner=True,kind='hist',diag_kind='kde')

In [ ]:
# Correlation matrix
corr = df.corr()
sns.heatmap(corr,cmap='bwr',vmin=-1,vmax=+1)

In [ ]:
# Data partitioning
# Input variables
X_names = df.columns
X_names = X_names.drop('plant.filters.product.iron.comp')
X_names = X_names.drop('plant.filters.product.silica.comp')
print(X_names)
# Output variables
Y_name = 'plant.filters.product.silica.comp'
# Data frames
X = df[X_names].copy()
Y = df[Y_name].rolling('24h').median().copy()
# Training (and validation) and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,shuffle=True)

# ⚠️⚠️⚠️ WARNING ⚠️⚠️⚠️
Shuffling samples for training and testing data sets **is not recommended** for building predictive models from time series data, given data leakage and process plant time variance! Shuffling is typically the default in many machine learning packages, please ensure to explicitly check and correct. `shuffle=True` is used here to demonstrate the disparity in results in comparison to `shuffle=False`.

In [ ]:
# Understand data partitioning
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(Y_train,'b.',label='Training data')
ax.plot(Y_test,'r.',label='Testing data')
ax.legend()

In [ ]:
# Preprocessing: Scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
# Linear model
linear_model = LinearRegression()

In [ ]:
# Tree model
tree_model = DecisionTreeRegressor(min_samples_leaf=50)

In [ ]:
# Boosted ensemble
boost_model = GradientBoostingRegressor(n_estimators=50)

In [ ]:
# Cross-validation
linear_model_cv = cross_val_score(linear_model,X_train_scaled,Y_train,scoring='r2',cv=5)
tree_model_cv = cross_val_score(tree_model,X_train_scaled,Y_train,scoring='r2',cv=5)
boost_model_cv = cross_val_score(boost_model,X_train_scaled,Y_train,scoring='r2',cv=5)

In [ ]:
# Model selection
print(f'Linear model average R squared: {linear_model_cv.mean():0.2f}')
print(f'Tree model average R squared: {tree_model_cv.mean():0.2f}')
print(f'Boost model average R squared: {boost_model_cv.mean():0.2f}')

In [ ]:
# Fit best model on training and validation data
boost_model.fit(X_train_scaled,Y_train)

In [ ]:
# Model performance on test data
X_test_scaled = scaler.transform(X_test)
Y_test_pred = boost_model.predict(X_test_scaled)
r2_test = r2_score(y_true=Y_test,y_pred=Y_test_pred)
print(f'Boost model test R squared: {r2_test:0.2f}')

In [ ]:
# Model performance on all data
df['predicted.plant.filters.product.silica.comp'] = boost_model.predict(scaler.transform(df[X_names]))
df['Train/test data'] = 'Train'
df.loc[X_test.index,'Train/test data'] = 'Test'
print(f'Boost model average R squared: {r2_score(df['plant.filters.product.silica.comp'],df['predicted.plant.filters.product.silica.comp']):0.2f}')

In [ ]:
# Time series plot of predictions
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(df.loc[df['Train/test data']=='Train','plant.filters.product.silica.comp'],'k.-',label='Training data, true value',alpha=0.5)
ax.plot(df.loc[df['Train/test data']=='Test','plant.filters.product.silica.comp'],'k*-',label='Testing data, true value',alpha=0.5)
ax.plot(df.loc[df['Train/test data']=='Train','predicted.plant.filters.product.silica.comp'],'.-',color='darkblue',label='Training data, predicted value',alpha=0.5)
ax.plot(df.loc[df['Train/test data']=='Test','predicted.plant.filters.product.silica.comp'],'*-',color='darkred',label='Testing data, predicted value',alpha=0.5)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))
ax.set_title('Time series plot of predictions')
ax.set_xlabel('Time')
ax.set_ylabel('Output variable')

In [ ]:
# Parity plot
fig, ax = plt.subplots(figsize=(6,6))
ax.plot(df.loc[df['Train/test data']=='Train','plant.filters.product.silica.comp'],
        df.loc[df['Train/test data']=='Train','predicted.plant.filters.product.silica.comp'],
        '.',color='darkblue',label='Training data',alpha=0.5)
ax.plot(df.loc[df['Train/test data']=='Test','plant.filters.product.silica.comp'],
        df.loc[df['Train/test data']=='Test','predicted.plant.filters.product.silica.comp'],
        '*',color='darkred',label='Testing data',alpha=0.5)
min_y = df['plant.filters.product.silica.comp'].min()
max_y = df['plant.filters.product.silica.comp'].max()
ax.plot([min_y, max_y],[min_y, max_y],color='k',label='Parity line')
ax.legend(loc='upper left',bbox_to_anchor=(1,1))
ax.set_title('Parity plot')
ax.set_xlabel('True values')
ax.set_ylabel('Predicted values')

# 🎯 Practice Points
- Based on the correlation matrix information, isualize different variables in time series plots and pair plots.
- Change the downsampling period, and observe the effect on the model result. What are the advantages and disadvantages of small and large downsampling perods?
- Limit the input data X to fewer variables, e.g., by only including one representitive variable from a correlated group. See how the reduced input set affects the model performance.
- Vary the training fraction, and observe the effect on the training and testing prediction performance.